# Chapter 3 - Interactivity

We'll be working exclusively in `app.py` for this set of exercises,, **but we have not provided app.py for you this time**. Create it, and then make sure your environment is `dv-env` and your directory is the same as this notebook.

In [ ]:
# Basic Scatter Plot

import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

df = pd.read_csv("car_sales.csv")

app = dash.Dash(__name__)

fig = px.scatter(df, x="Price_in_thousands", y="Sales_in_thousands", color="Vehicle_type")

app.layout = html.Div([
    html.H1("Car Sales Dashboard"),
    dcc.Graph(figure=fig)
])

### Adding A Dropdown

In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

cars = pd.read_csv("car-sales.csv")

app = dash.Dash(__name__)

fig = px.scatter(
    cars,
    x="Horsepower",
    y="Fuel Efficiency",
    color="Price In Thousands",
)

options = [{"label": manufacturer, "value": manufacturer} for manufacturer in cars["Manufacturer"].unique()]
options.insert(0, {"label": "All Manufacturers", "value": "All"})

app.layout = html.Div([
    html.H1("Fuel Efficiency Vs. Horsepower"),
    dcc.Dropdown(
        id="manufacturer-dropdown",
        options=options,
        value="All"
    ),

    dcc.Graph(id="sales-graph")
])

@app.callback(
    Output("sales-graph", "figure"),
    Input("manufacturer-dropdown", "value")
)
def update_graph(selected_manufacturer):
    filtered_cars = cars[cars["Manufacturer"] == selected_manufacturer]
    if selected_manufacturer == "All":
        filtered_cars = cars

    fig = px.scatter(
        filtered_cars,
        x="Horsepower",
        y="Fuel Efficiency",
        color="Price In Thousands",
        hover_name="Model"
    )

    return fig

app.run(debug=True)

### Adding A Slider

In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

cars = pd.read_csv("car-sales.csv")

app = dash.Dash(__name__)

fig = px.scatter(
    cars,
    x="Price In Thousands",
    y="Sales In Thousands",
    color="Manufacturer",
)

options = [{"label": manufacturer, "value": manufacturer} for manufacturer in cars["Manufacturer"].unique()]
options.insert(0, {"label": "All Manufacturers", "value": "All"})

app.layout = html.Div([
    html.H1("Fuel Efficiency Vs. Horsepower"),
    dcc.Dropdown(
        id="manufacturer-dropdown",
        options=options,
        value="All"
    ),

    dcc.RangeSlider(
        id="engine-size-slider",
        min=int(cars["Engine Size"].min()),
        max=int(cars["Engine Size"].max()),
        step=1,
        value=[int(cars["Engine Size"].min()), int(cars["Engine Size"].max())],
        marks={i: str(i) for i in range(-1, int(cars["Engine Size"].max()) + 1, 100)},
        tooltip={"placement": "bottom"}
    ),

    dcc.Graph(id="sales-graph")
])

@app.callback(
    Output("sales-graph", "figure"),
    [
        Input("manufacturer-dropdown", "value"),
        Input("engine-size-slider", "value")
    ]
)
def update_graph(selected_manufacturer, engine_size_range):
    filtered_cars = cars[
        (cars["Manufacturer"] == selected_manufacturer) &
        (cars["Engine Size"].between(engine_size_range[0], engine_size_range[1]))
    ]

    if selected_manufacturer == "All":
        filtered_cars = cars[ cars["Engine Size"].between(engine_size_range[0], engine_size_range[1]) ]

    fig = px.scatter(
        filtered_cars,
        x="Horsepower",
        y="Fuel Efficiency",
        color="Price In Thousands",
        hover_name="Model"
    )

    return fig

app.run(debug=True)

In [ ]:
# Add a hover card for each car.

@app.callback(
    Output("info-panel", "children"),
    Input("sales-graph", "hoverData")
)
def display_hover_info(hoverData):
    if hoverData:
        point = hoverData["points"][0]
        model = point["hovertext"]
        row = cars[cars["Model"] == model].iloc[0]
        return html.Div([
            html.H3(f"{row['Manufacturer']} {row['Model']}"),
            html.P(f"Horsepower: {row['Horsepower']}"),
            html.P(f"Fuel Efficiency: {row['Fuel Efficiency']}"),
            html.P(f"Engine Size: {row['Engine Size']}L"),
        ])
